In [ ]:
import os

# Create directories if they don't exist
dirs = ["data_5k", "scripts_5k", "results_5k", "figures"]
for dir in dirs:
    os.makedirs(dir, exist_ok=True)
print("Directories created successfully: data_5k, scripts_5k, results_5k, figures_5k.")


In [ ]:
import scanpy as sc

# Load the 5k dataset
adata_5k = sc.read_h5ad("data_5k/5k_pbmc_10x.h5ad")

# Save raw data for future use
adata_5k.raw = adata_5k

# Save the loaded data for checkpoint
adata_5k.write("results_5k/adata_raw_5k.h5ad")

print("5k dataset loaded and raw data saved successfully.")


In [ ]:
# Quality control and filtering
sc.pp.filter_cells(adata_5k, min_genes=200)
sc.pp.filter_genes(adata_5k, min_cells=3)
print("Quality control: Filtered cells with fewer than 200 genes and genes detected in fewer than 3 cells.")

# Annotate mitochondrial genes
adata_5k.var['mt'] = adata_5k.var_names.str.startswith('MT-')

# Compute QC metrics (e.g., percentage of mitochondrial genes)
sc.pp.calculate_qc_metrics(adata_5k, qc_vars=['mt'], inplace=True)


# Normalize the data
sc.pp.normalize_total(adata_5k, target_sum=1e4)
sc.pp.log1p(adata_5k)
print("Normalization complete: Data normalized to 10,000 counts per cell and log-transformed.")

# Identify highly variable genes
sc.pp.highly_variable_genes(adata_5k, min_mean=0.0125, max_mean=3, min_disp=0.5)
print("Highly variable genes identified with specified parameters.")

hvg_5k = sum(adata_5k.var['highly_variable'])
print(f"Number of highly variable genes in the 5k dataset: {hvg_5k}")

# Save preprocessing results
adata_5k.write("results_5k/adata_preprocessed_5k.h5ad")
print("Preprocessed data saved successfully.")


In [ ]:
# Scale the data to have mean 0 and variance 1 for each gene
sc.pp.scale(adata_5k, zero_center=True, max_value=10)

# PCA
sc.pp.pca(adata_5k, n_comps=50)
sc.pl.pca(adata_5k, save="pca_5k.png")
print("PCA complete: Saved PCA plot in the 'figures' folder.")

# Neighbor graph construction
sc.pp.neighbors(adata_5k, n_neighbors=10, n_pcs=40)
print("Neighbor graph construction complete.")

# Clustering using Leiden algorithm
sc.tl.leiden(adata_5k)
print("Clustering complete: Performed Leiden clustering.")

# Save clustering results
adata_5k.write("results_5k/adata_clustered_5k.h5ad")
print("Clustering results saved successfully in 'results_5k' folder.")


In [ ]:
# UMAP for visualization
sc.tl.umap(adata_5k)
sc.pl.umap(adata_5k, color=["leiden"], save="umap_5k.png")
print("UMAP complete: Saved UMAP plot in the 'figures' folder.")

# Plot UMAP with marker genes
sc.pl.umap(adata_5k, color=['CST3', 'NKG7', 'PPBP', 'MS4A1'], save='_markers_5k.png')

print("Plots saved in the 'figures' directory.")


In [ ]:
# Perform differential expression analysis
sc.tl.rank_genes_groups(adata_5k, 'leiden', method='t-test')
print("Differential expression analysis complete.")

# Save marker gene ranking plot
sc.pl.rank_genes_groups(adata_5k, save="top_genes_5k.png")
print("Saved marker gene plot in the 'figures_5k' folder.")

# Save gene ranking results
import pandas as pd

# Convert results to DataFrame and save
gene_ranking = pd.DataFrame(adata_5k.uns['rank_genes_groups']['names'])
gene_ranking.to_csv("results_5k/rank_genes_groups_5k.csv")
print("Marker gene ranking results saved as 'rank_genes_groups_5k.csv' in the 'results_5k' folder.")
